In [13]:
import sys
import time
import warnings
import numpy as np
from numpy.linalg import norm

from studies.utils import *
from pyROMs.pod import POD

warnings.filterwarnings('ignore')

Parse the Data

In [15]:
print("Loading and formating the data...")
t_start = time.time()
if problem == 'infinite_slab':
    dataset = get_data(problem, case, study)
else:
    dataset = get_data(problem, study)
t_end = time.time()
print(f"Loading the data took {t_end - t_start:3f} s")

Loading and formating the data...
Loading the data took 13.332585 s


Parameter Definitions

In [19]:
problem = 'three_group_sphere'
case = 0
study = 3

var = None
if problem != 'three_group_sphere':
    var = 'power_density'

test_size = 0.2
interior_only = False
seed = 12

tau = 1.0e-8
epsilon = 200.0

Split the Data

In [16]:
splits = dataset.train_test_split(variables=var,
                                  test_size=test_size, seed=seed,
                                  interior_only=interior_only)
X_train, X_test, Y_train, Y_test = splits

Perform the Interpolation Study

In [17]:
print(f"Starting the interpolation study...")

# Define the different interpolants
interps = ['rbf_linear', 'rbf_thin_plate_spline',
           'rbf_cubic', 'rbf_quintic', 'rbf_gaussian',
           'linear', 'nearest']

# Loop over interpolants
t_start = time.time()
errors = {'mean': [], 'max': [], 'min': []}
for interp in interps:

    # Construct POD model, predict test data
    pod = POD(svd_rank=0)
    pod.fit(X_train, Y_train, interp, epsilon=epsilon)

    # Predict results
    X_pred = pod.predict(Y_test)

    # Format datasets, compute errors
    X_pred = dataset.unstack_simulation_vector(X_pred)
    X_test = dataset.unstack_simulation_vector(X_test)
    error_i = np.zeros(len(X_test))
    for i in range(len(X_test)):
        error_i[i] = norm(X_test[i]-X_pred[i])/norm(X_test[i])
    errors['mean'].append(np.mean(error_i))
    errors['max'].append(np.max(error_i))
    errors['min'].append(np.min(error_i))
t_end = time.time()
print(f"Finished interplant study in {t_end-t_start:.3f} s.")
print(f"Number of Snapshots:\t{pod.n_snapshots}")
print(f"Number of Validations:\t{len(X_test)}")
print(f"Number of POD Modes:\t{pod.n_modes}")

Starting the interpolation study...
Finished interplant study in 0.501 s.
Number of Snapshots:	100
Number of Validations:	25
Number of POD Modes:	48


Prin LaTeX Table

In [18]:
msg = "\\begin{tabular}{|c|c|c|c|}" \
      "\n\t\hline" \
      "\n\t\\textbf{Interpolation Method} & \\textbf{Mean Error} & " \
      "\\textbf{Max Error} & \\textbf{Min Error} \\\\ \hline"
for i in range(len(interps)):
    interp = interps[i].split('_')
    if interp[0] == 'rbf':
        interp[0] = interp[0].upper()
        for w in range(len(interp[1:])):
            interp[w+1] = interp[w+1].capitalize()
        interp = " ".join(interp[1:]) + " RBF"
        if "Gaussian" in interp:
            interp += f", $\\epsilon$ = {epsilon:.2e}"
    else:
        interp = interp[0].capitalize()
        if interp == "Nearest":
            interp += " Neighbor"
        elif interp == "Cubic":
            interp += " Spline"

    msg += f"\n\t\hline {interp} & " \
           f"{errors['mean'][i]:.3e} & " \
           f"{errors['max'][i]:.3e} & " \
           f"{errors['min'][i]:.3e} \\\\"
msg += "\n\t\hline\n\\end{tabular}"
print(msg)

\begin{tabular}{|c|c|c|c|}
	\hline
	\textbf{Interpolation Method} & \textbf{Mean Error} & \textbf{Max Error} & \textbf{Min Error} \\ \hline
	\hline Linear RBF & 3.196e-02 & 1.238e-01 & 5.747e-04 \\
	\hline Thin Plate Spline RBF & 3.077e-03 & 1.059e-02 & 1.365e-04 \\
	\hline Cubic RBF & 2.972e-03 & 1.640e-02 & 1.588e-04 \\
	\hline Quintic RBF & 1.650e-03 & 5.377e-03 & 2.846e-04 \\
	\hline Gaussian RBF, $\epsilon$ = 2.00e+02 & 2.612e-03 & 1.289e-02 & 3.005e-06 \\
	\hline Linear & nan & nan & nan \\
	\hline Nearest Neighbor & 1.581e-01 & 2.465e-01 & 3.531e-02 \\
	\hline
\end{tabular}
